# Improving the annotation of metabolite transporters

First off, we are importing packages that might come in handy.

In [1]:
import cobra
import numpy as np
import scipy as sp
import pandas as pd
import requests
from io import StringIO
from Bio import SeqIO

Importing most recenet TC numbers, corresponding CHEBI ID of substrates, uniprot ID and GO terms from TCDB

In [18]:
tc_chebi_url = "https://www.tcdb.org/cgi-bin/substrates/getSubstrates.py"
uniprot_tc_url = "https://www.tcdb.org/cgi-bin/projectv/public/acc2tcid.py"
go_tc_url = "https://www.tcdb.org/cgi-bin/projectv/public/go.py"
fasta_tcdb_url = "https://www.tcdb.org/public/tcdb"

def fetch_data(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

def parse_data(tc_chebi_text, uniprot_tc_text, go_tc_text, fasta_tcdb_text):
    # TC-CHEBI
    tc_chebi_lines = tc_chebi_text.strip().split("\n")
    tc_chebi_data = []
    for line in tc_chebi_lines:
        tc_number, chebi_ids = line.split("\t")
        chebi_id_list = [id.split(";")[0].replace("CHEBI:", "") for id in chebi_ids.split("|")]
        tc_chebi_data.append([tc_number, chebi_id_list])

    df_chebi = pd.DataFrame(tc_chebi_data, columns=["TC Number", "CHEBI IDs"])

    # UniProt-TC
    uniprot_tc_lines = uniprot_tc_text.strip().split("\n")
    uniprot_tc_data = [line.split("\t") for line in uniprot_tc_lines]
    df_uniprot = pd.DataFrame(uniprot_tc_data, columns=["UniProt ID", "TC Number"])

    # GO-TC
    go_tc_lines = go_tc_text.strip().split("\n")
    go_tc_data = [line.split("\t")[:2] for line in go_tc_lines]
    df_go = pd.DataFrame(go_tc_data, columns=["GO Term", "TC Number"])

    # FASTA-TC
    fasta_io = StringIO(fasta_tcdb_text)
    tc_data = []
    for record in SeqIO.parse(fasta_io, "fasta"):
        header = record.description

        uniprot = header.split("|")[2]
        tc_number = header.split("|")[3].split()[0]
        sequence = str(record.seq)
        tc_data.append([tc_number, uniprot, sequence])
    df_fasta = pd.DataFrame(tc_data, columns=["TC Number", "UniProt ID", "AA Sequence"])

    return df_chebi, df_uniprot, df_go, df_fasta

In [19]:
tc_chebi_text = fetch_data(tc_chebi_url)
uniprot_tc_text = fetch_data(uniprot_tc_url)
go_tc_text = fetch_data(go_tc_url)
fasta_tcdb_text = fetch_data(fasta_tcdb_url)

In [23]:
df_chebi, df_uniprot, df_go, df_fasta = parse_data(tc_chebi_text, uniprot_tc_text, go_tc_text, fasta_tcdb_text)
df_chebi.to_csv("tcdb_chebis.csv", index=False)
df_merged = pd.merge(df_chebi, df_fasta, on="TC Number", how="left")
df_merged = df_merged.explode('CHEBI IDs')
df_merged = pd.merge(df_merged, df_go[["TC Number", "GO Term"]], on="TC Number", how="left")
df_merged = df_merged.drop_duplicates()
df_merged.to_csv("tcdb_data_combined.csv", index=False)
df_merged

,TC Number,CHEBI IDs,UniProt ID,AA Sequence,GO Term
0,1.A.77.3.11,3473,S0WRQ6,MSVSLLSRNVARKLLLVQLLAVIASGLLFSLKDPFWGVSALSGGLA...,NaN
1,3.A.1.105.15,23888,C6EIY9,MSNPILSWRRVRALCVKETRQIVRDPSSWLIAVVIPLLLLFIFGYG...,NaN
2,3.A.1.105.15,23888,P0A9U1,MNDAVITLNGLEKRFPGMDKPAVAPLDCTIHAGYVTGLVGPDGAGK...,NaN
3,3.A.1.105.15,23888,P0AFP9,MFHRLWTLIRKELQSLLREPQTRAILILPVLIQVILFPFAATLEVT...,NaN
4,3.A.1.105.15,23888,P75777,MMKKPVVIGLAVVVLAAVVAGGYWWYQSRQDNGLTLYGNVDIRTVN...,NaN
...,...,...,...,...,...
154251,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005886
154252,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005524
154253,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005516
154254,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0006955


In [9]:
df_merged

,TC Number,CHEBI IDs,UniProt ID,GO Term,AA Sequence,ChEBI Primary IDs
0,1.C.85.1.1,25367,P60022,GO:0005576,MRTSYLLLFTLCLLLSEMASGGNFLTGLGHRSDHYNCVSSGGQCLY...,25367
1,1.C.85.1.1,25367,P60022,GO:0006935,MRTSYLLLFTLCLLLSEMASGGNFLTGLGHRSDHYNCVSSGGQCLY...,25367
2,1.C.85.1.1,25367,P60022,GO:0042742,MRTSYLLLFTLCLLLSEMASGGNFLTGLGHRSDHYNCVSSGGQCLY...,25367
3,1.C.85.1.1,25367,P60022,GO:0007186,MRTSYLLLFTLCLLLSEMASGGNFLTGLGHRSDHYNCVSSGGQCLY...,25367
4,1.C.85.1.1,25367,P60022,GO:0045087,MRTSYLLLFTLCLLLSEMASGGNFLTGLGHRSDHYNCVSSGGQCLY...,25367
...,...,...,...,...,...,...
1757696,1.A.8.13.3,5585,D7VF10,NaN,MRKYLAEFLGTFMLVFLGTATVVIAKGDVLAIGLAFGLAITVSAYA...,15377
1757697,1.A.8.13.3,5453,D7VF10,NaN,MRKYLAEFLGTFMLVFLGTATVVIAKGDVLAIGLAFGLAITVSAYA...,16016
1757698,1.A.8.13.3,5448,D7VF10,NaN,MRKYLAEFLGTFMLVFLGTATVVIAKGDVLAIGLAFGLAITVSAYA...,17754
1757699,2.A.113.1.11,23337,G8N7A2,NaN,MMEWASYVMLAIFLGMRHGMDSDHVAAIADMVGSESRRRQQLALGI...,48828


Making an easy way to convert from primary/secondaryy ChEBI ID to primary ChEBI ID only.

In [24]:
prim_sec_chebi = pd.read_csv("chebi_data/primary_secondary_chebi_ids.tsv", sep="\t")

secondary_to_primary =  {}

for _, row in prim_sec_chebi.iterrows():
    primary_id = int(row["Primary_CHEBI_ID"])
    secondary_ids = eval(row["Secondary_CHEBI_IDs"])


    for s_id in secondary_ids:
        secondary_to_primary[s_id] = primary_id

def get_primary_id(chebi_id):
    chebi_id = int(chebi_id)
    return secondary_to_primary.get(str(chebi_id), chebi_id)

Convert all ChEBI IDs to primary IDs for the TCDB df

In [25]:
df_merged["CHEBI IDs"] = pd.to_numeric(df_merged["CHEBI IDs"], errors="raise")
df_merged.loc[:, "ChEBI Primary IDs"] = df_merged["CHEBI IDs"].apply(get_primary_id)
df_merged

,TC Number,CHEBI IDs,UniProt ID,AA Sequence,GO Term,ChEBI Primary IDs
0,1.A.77.3.11,3473,S0WRQ6,MSVSLLSRNVARKLLLVQLLAVIASGLLFSLKDPFWGVSALSGGLA...,NaN,3473
1,3.A.1.105.15,23888,C6EIY9,MSNPILSWRRVRALCVKETRQIVRDPSSWLIAVVIPLLLLFIFGYG...,NaN,23888
2,3.A.1.105.15,23888,P0A9U1,MNDAVITLNGLEKRFPGMDKPAVAPLDCTIHAGYVTGLVGPDGAGK...,NaN,23888
3,3.A.1.105.15,23888,P0AFP9,MFHRLWTLIRKELQSLLREPQTRAILILPVLIQVILFPFAATLEVT...,NaN,23888
4,3.A.1.105.15,23888,P75777,MMKKPVVIGLAVVVLAAVVAGGYWWYQSRQDNGLTLYGNVDIRTVN...,NaN,23888
...,...,...,...,...,...,...
154251,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005886,29103
154252,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005524,29103
154253,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005516,29103
154254,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0006955,29103


Creating the filter that is applied to only find accurate ChEBIs. See chebi_data/leaf_children.ipynb for more info regarding the filter choice.

The method used, is Method 2. This gives best performance (includes the most IDs where info is obtainable).
This filter aims to only leave in leaf nodes, and parents that have no children present in the df from TCDB.

First loading in the hierarchy pr 2024-08-01.

In [56]:
df_hierarchy = pd.read_csv("chebi_data/chebiHierarchy.tsv", sep="\t")
df_hierarchy['child'] = df_hierarchy['child'].str.extract(r'CHEBI_(\d+)').astype(int)
df_hierarchy['parent'] = df_hierarchy['parent'].str.extract(r'CHEBI_(\d+)').astype(int)

all_primary_chebi_tcdb = set(df_merged["ChEBI Primary IDs"].unique())

df_hierarchy_prim = df_hierarchy.copy()
df_hierarchy_prim["child"] = df_hierarchy["child"].apply(get_primary_id)
df_hierarchy_prim["parent"] = df_hierarchy["parent"].apply(get_primary_id)

Finding all the parents from the original (TCDB) df that HAS a child listed in the df.

In [57]:
parents_w_children_in_tcdb = set(df_hierarchy_prim[df_hierarchy_prim["child"].isin(all_primary_chebi_tcdb)]["parent"])

chebi_ids_to_remove = parents_w_children_in_tcdb.intersection(all_primary_chebi_tcdb)

filtered_chebis = all_primary_chebi_tcdb - chebi_ids_to_remove

Applying the filter where there are only leaf nodes and parents without children, and reducing df_merged correspondingly.

In [58]:
df_filtered = df_merged[df_merged["ChEBI Primary IDs"].isin(filtered_chebis)]
df_filtered

,TC Number,CHEBI IDs,UniProt ID,go,go_label,AA Sequence,ChEBI Primary IDs
0,3.A.3.27.2,3473,B0STR2,NaN,NaN,MNKTNSQSTTTECDHCGNPIQLIRIESKLGDEPKVFCCEGCETVYS...,3473
1,5.B.7.2.1,10545,Q7NNK2,NaN,NaN,MVRANTPWLTTLAVLVAAYIVAALLETVTLILFRGHLQGQRLGEYY...,10545
2,2.A.1.2.67,8301,P53283,GO_0071944,cell periphery,MSDQESVVSFNSQNTSMVDVEGQQPQQYVPSKTNSRANQLKLTKTE...,8301
3,2.A.1.2.67,8301,P53283,GO_0000329,fungal-type vacuole membrane,MSDQESVVSFNSQNTSMVDVEGQQPQQYVPSKTNSRANQLKLTKTE...,8301
4,2.A.1.2.67,8301,P53283,GO_0005886,plasma membrane,MSDQESVVSFNSQNTSMVDVEGQQPQQYVPSKTNSRANQLKLTKTE...,8301
...,...,...,...,...,...,...,...
374372,2.A.6.2.27,29697,Q9I6X5,NaN,NaN,MSDARGAFHSKGRWSRMALPAILCAGLLVGCGAEPPAEEHVRVLAQ...,164200
374373,2.A.6.2.27,29697,Q9I6X6,NaN,NaN,MLRRLSLAAAVAAATGVAWAAQPTPLPTKTDLISVYKEAVDNNADL...,164200
374374,2.A.6.2.27,29697,Q9I6X6,NaN,NaN,MKGGFNLSDWALRHQSLVWYLMAVSLVMGVFSYLNLGREEDPSFAI...,164200
374375,2.A.6.2.27,29697,Q9I6X6,NaN,NaN,MKPFSLAGLFGFALLLSGCGDEPPPAPPRPVLTVTVKTLKNDDLGR...,164200


Now I want to append all the ChEBI information to the substrates. Charge, formula, MW, and SMILES, to be precise. But first I need to make sure it is the primary ChEBI ID that is in use for both dfs. To reduce mismatches.

In [26]:
chebi_df = pd.read_csv("chebi_data\chebiDf.tsv", sep="\t")
chebi_df["chebi"] = chebi_df["chebi"].str.extract(r"CHEBI_(\d+)").astype(int)
chebi_df = chebi_df.drop(columns=["inchi", "inchikey"])

chebi_df.loc[:, "chebi_primary"] = chebi_df["chebi"].apply(get_primary_id)

In [60]:
df_chebi_info = df_filtered.merge(chebi_df, left_on="ChEBI Primary IDs", right_on="chebi_primary", how="left")
df_chebi_info.rename(columns={"label":"chebi_label"}, inplace=True)
df_chebi_info

,TC Number,CHEBI IDs,UniProt ID,go,go_label,AA Sequence,ChEBI Primary IDs,charge,chebi,formula,chebi_label,mass,smiles,chebi_primary
0,3.A.3.27.2,3473,B0STR2,NaN,NaN,MNKTNSQSTTTECDHCGNPIQLIRIESKLGDEPKVFCCEGCETVYS...,3473,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.B.7.2.1,10545,Q7NNK2,NaN,NaN,MVRANTPWLTTLAVLVAAYIVAALLETVTLILFRGHLQGQRLGEYY...,10545,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.A.1.2.67,8301,P53283,GO_0071944,cell periphery,MSDQESVVSFNSQNTSMVDVEGQQPQQYVPSKTNSRANQLKLTKTE...,8301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.A.1.2.67,8301,P53283,GO_0000329,fungal-type vacuole membrane,MSDQESVVSFNSQNTSMVDVEGQQPQQYVPSKTNSRANQLKLTKTE...,8301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.A.1.2.67,8301,P53283,GO_0005886,plasma membrane,MSDQESVVSFNSQNTSMVDVEGQQPQQYVPSKTNSRANQLKLTKTE...,8301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253386,2.A.6.2.27,29697,Q9I6X5,NaN,NaN,MSDARGAFHSKGRWSRMALPAILCAGLLVGCGAEPPAEEHVRVLAQ...,164200,0.0,164200.0,C12H7Cl3O2,triclosan,289.542,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,164200.0
253387,2.A.6.2.27,29697,Q9I6X6,NaN,NaN,MLRRLSLAAAVAAATGVAWAAQPTPLPTKTDLISVYKEAVDNNADL...,164200,0.0,164200.0,C12H7Cl3O2,triclosan,289.542,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,164200.0
253388,2.A.6.2.27,29697,Q9I6X6,NaN,NaN,MKGGFNLSDWALRHQSLVWYLMAVSLVMGVFSYLNLGREEDPSFAI...,164200,0.0,164200.0,C12H7Cl3O2,triclosan,289.542,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,164200.0
253389,2.A.6.2.27,29697,Q9I6X6,NaN,NaN,MKPFSLAGLFGFALLLSGCGDEPPPAPPRPVLTVTVKTLKNDDLGR...,164200,0.0,164200.0,C12H7Cl3O2,triclosan,289.542,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,164200.0


Alternatively, without using the filter at all....

In [27]:
df_chebi_info = df_merged.merge(chebi_df, left_on="ChEBI Primary IDs", right_on="chebi_primary", how="left")
df_chebi_info.rename(columns={"label":"chebi_label"}, inplace=True)
df_chebi_info

,TC Number,CHEBI IDs,UniProt ID,AA Sequence,GO Term,ChEBI Primary IDs,charge,chebi,formula,chebi_label,mass,smiles,chebi_primary
0,1.A.77.3.11,3473,S0WRQ6,MSVSLLSRNVARKLLLVQLLAVIASGLLFSLKDPFWGVSALSGGLA...,NaN,3473,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.A.1.105.15,23888,C6EIY9,MSNPILSWRRVRALCVKETRQIVRDPSSWLIAVVIPLLLLFIFGYG...,NaN,23888,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.A.1.105.15,23888,P0A9U1,MNDAVITLNGLEKRFPGMDKPAVAPLDCTIHAGYVTGLVGPDGAGK...,NaN,23888,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.A.1.105.15,23888,P0AFP9,MFHRLWTLIRKELQSLLREPQTRAILILPVLIQVILFPFAATLEVT...,NaN,23888,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.A.1.105.15,23888,P75777,MMKKPVVIGLAVVVLAAVVAGGYWWYQSRQDNGLTLYGNVDIRTVN...,NaN,23888,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94851,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005886,29103,1.0,29103.0,K,potassium(1+),39.0983,[K+],29103.0
94852,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005524,29103,1.0,29103.0,K,potassium(1+),39.0983,[K+],29103.0
94853,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005516,29103,1.0,29103.0,K,potassium(1+),39.0983,[K+],29103.0
94854,1.A.4.5.4,8345,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0006955,29103,1.0,29103.0,K,potassium(1+),39.0983,[K+],29103.0


Worth noting that e- with ChEBI 10545 is missing. Should be inserted manually with all its properties. Change is->was when done

In [28]:
# df_chebi_info[df_chebi_info["CHEBI IDs"] == 1]
df_missing_chebis = df_chebi_info[df_chebi_info['chebi'].isna()]
print(f"There are {len(df_missing_chebis)} ChEBI IDs that cannot be connected to any info, i.e. are too broad, like 'molecule' or 'polypeptide'.")

There are 33710 ChEBI IDs that cannot be connected to any info, i.e. are too broad, like 'molecule' or 'polypeptide'.


Next up, removing all instances from the df where there is no info to get on the ChEBI substrate

In [52]:
valid_chebis = chebi_df["chebi"]
df_chebi_info_filtered = df_chebi_info[df_chebi_info["ChEBI Primary IDs"].isin(valid_chebis)]
# Some quick deletions and rearranging of the df, to make it more tidy
df_chebi_info_filtered = df_chebi_info_filtered.drop(columns=["chebi", "chebi_primary"])
columns = list(df_chebi_info_filtered.columns)
current_index = columns.index("CHEBI IDs")
new_index = current_index + 3
columns.pop(current_index)
columns.insert(new_index, "CHEBI IDs")
df_chebi_info_filtered = df_chebi_info_filtered[columns]

df_chebi_info_filtered = df_chebi_info_filtered.reset_index(drop=True)
df_chebi_info_filtered.to_csv("tcdb_data_chebi.csv", index=False)
df_chebi_info_filtered

,TC Number,UniProt ID,AA Sequence,GO Term,CHEBI IDs,ChEBI Primary IDs,charge,formula,chebi_label,mass,smiles
0,4.A.1.1.2,P09323,MNILGFFQRLGRALQLPIAVLPVAALLLRFGQPDLLNVAFIAQAGG...,GO:0016021,28009,28009,0.0,C8H15NO6,N-acetyl-beta-D-glucosamine,221.2078,CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](O)[C@@H]1O
1,4.A.1.1.2,P09323,MNILGFFQRLGRALQLPIAVLPVAALLLRFGQPDLLNVAFIAQAGG...,GO:0019866,28009,28009,0.0,C8H15NO6,N-acetyl-beta-D-glucosamine,221.2078,CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](O)[C@@H]1O
2,4.A.1.1.2,P09323,MNILGFFQRLGRALQLPIAVLPVAALLLRFGQPDLLNVAFIAQAGG...,GO:0005886,28009,28009,0.0,C8H15NO6,N-acetyl-beta-D-glucosamine,221.2078,CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](O)[C@@H]1O
3,4.A.1.1.2,P09323,MNILGFFQRLGRALQLPIAVLPVAALLLRFGQPDLLNVAFIAQAGG...,GO:0016301,28009,28009,0.0,C8H15NO6,N-acetyl-beta-D-glucosamine,221.2078,CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](O)[C@@H]1O
4,4.A.1.1.2,P09323,MNILGFFQRLGRALQLPIAVLPVAALLLRFGQPDLLNVAFIAQAGG...,GO:0008982,28009,28009,0.0,C8H15NO6,N-acetyl-beta-D-glucosamine,221.2078,CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](O)[C@@H]1O
...,...,...,...,...,...,...,...,...,...,...,...
61141,1.A.4.5.4,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005886,8345,29103,1.0,K,potassium(1+),39.0983,[K+]
61142,1.A.4.5.4,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005524,8345,29103,1.0,K,potassium(1+),39.0983,[K+]
61143,1.A.4.5.4,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0005516,8345,29103,1.0,K,potassium(1+),39.0983,[K+]
61144,1.A.4.5.4,Q8TD43,MVVPEKEQSWIPKIFKKKTCTTFIVDSTDPGGTLCQCGRPRTAHPA...,GO:0006955,8345,29103,1.0,K,potassium(1+),39.0983,[K+]


In [80]:
# df_chebi_info_filtered.head(n=20)
# rows = [i for i in range(56264, 56270)]
df_chebi_info_filtered.iloc[[90,91,97,98,118,119],[0,1,3,4,6,7,8,9,10]]

,TC Number,UniProt ID,GO Term,CHEBI IDs,charge,formula,chebi_label,mass,smiles
90,1.A.75.1.1,Q92508,GO:0005789,9175,1.0,Na,sodium(1+),22.98977,[Na+]
91,1.A.75.1.1,Q92508,GO:0033116,9175,1.0,Na,sodium(1+),22.98977,[Na+]
97,1.A.75.1.1,Q92508,GO:0005789,8345,1.0,K,potassium(1+),39.09830,[K+]
98,1.A.75.1.1,Q92508,GO:0033116,8345,1.0,K,potassium(1+),39.09830,[K+]
118,1.A.75.1.1,Q92508,GO:0005789,35264,1.0,C4H12N,tetramethylammonium,74.14482,C[N+](C)(C)C
119,1.A.75.1.1,Q92508,GO:0033116,35264,1.0,C4H12N,tetramethylammonium,74.14482,C[N+](C)(C)C
